In [1]:
import numpy as np
from keras.models import Model
from keras.layers import LSTM,Dense,Input
from sklearn.model_selection import train_test_split
import os
import keras
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"

Using TensorFlow backend.


In [2]:
input_data=[]
output_data = []

file = open('./fra.txt', 'r', encoding='utf-8-sig')

for idx, line in enumerate(file):
    if idx == 100000:
        break
    line = line.split('\t')
    input_data.append(line[0])
    output_data.append('\t' + line[1] + '\n')

In [3]:
batch_size = 64
input_max_seq = max([len(text) for text in input_data])
output_max_seq = max([len(text) for text in output_data])
input_voc_size = 0
output_voc_size = 0
embedding_size = 256

In [4]:
input_voc = set()
output_voc = set()
for ii,[input_text,output_text] in enumerate(zip(input_data,output_data)):
    for word in input_text:
        input_voc.add(word)
    for word in output_text:
        output_voc.add(word)

In [5]:
input_voc = sorted(input_voc)
output_voc = sorted(output_voc)
input_voc_size = len(input_voc)
output_voc_size = len(output_voc)
data_size = len(input_data)

In [6]:
input_to_num = {name:num for num,name in enumerate(input_voc)}
output_to_num = {name:num for num,name in enumerate(output_voc)}

In [7]:
encoder_input_datas = np.zeros((data_size,input_max_seq,input_voc_size),dtype = 'float32')
decoder_input_datas= np.zeros((data_size,output_max_seq,output_voc_size),dtype = 'float32')
decoder_output_datas = np.zeros((data_size,output_max_seq,output_voc_size),dtype = 'float32')

In [8]:
for ii,[input_text,output_text] in enumerate(zip(input_data,output_data)):
    for t,text in enumerate(input_text):
        encoder_input_datas[ii,t,input_to_num[text]] = 1.
    for t,text in enumerate(output_text):
        decoder_input_datas[ii,t,output_to_num[text]] = 1.
        if t > 0:
            decoder_output_datas[ii,t-1,output_to_num[text]] = 1.

In [9]:
encoder_inputs = Input((input_max_seq,input_voc_size))
encoder_lstm = LSTM(embedding_size,return_state=True)
encodet_outputs,state_h,state_c = encoder_lstm(encoder_inputs)
encoder_state = [state_h,state_c]

decoder_inputs = Input((None,output_voc_size))
decoder_lstm = LSTM(embedding_size,return_sequences= True, return_state= True)
decoder_outputs,_,_ = decoder_lstm(decoder_inputs,initial_state = encoder_state)
decoder_dense = Dense(output_voc_size,activation="softmax")

decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 79)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 106)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 344064      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  371712      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [10]:
model = keras.utils.multi_gpu_model(model, gpus=3)

model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

es = keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', verbose=1, patience=3, restore_best_weights=True)

model.fit([encoder_input_datas,decoder_input_datas],decoder_output_datas,
          batch_size = batch_size,
          epochs = 50,validation_split=0.2, callbacks=[es])

Train on 80000 samples, validate on 20000 samples
Epoch 1/50
80000/80000 [==============================] - 205s 3ms/step - loss: 0.6643 - val_loss: 0.6974
Epoch 2/50
80000/80000 [==============================] - 199s 2ms/step - loss: 0.4549 - val_loss: 0.5678
Epoch 3/50
80000/80000 [==============================] - 199s 2ms/step - loss: 0.3867 - val_loss: 0.5025
Epoch 4/50
80000/80000 [==============================] - 199s 2ms/step - loss: 0.3471 - val_loss: 0.4707
Epoch 5/50
80000/80000 [==============================] - 198s 2ms/step - loss: 0.3215 - val_loss: 0.4467
Epoch 6/50
80000/80000 [==============================] - 199s 2ms/step - loss: 0.3034 - val_loss: 0.4303
Epoch 7/50
80000/80000 [==============================] - 201s 3ms/step - loss: 0.2898 - val_loss: 0.4203
Epoch 8/50
80000/80000 [==============================] - 202s 3ms/step - loss: 0.2786 - val_loss: 0.4128
Epoch 9/50
80000/80000 [==============================] - 202s 3ms/step - loss: 0.2694 - val_loss: 0.4

In [11]:
encoder_model = Model(encoder_inputs,encoder_state)
decoder_input_h = Input(shape = (embedding_size,))
decoder_input_c = Input(shape = (embedding_size,))
decoder_input_state = [decoder_input_h,decoder_input_c]

decoder_outputs,state_h,state_c = decoder_lstm(decoder_inputs,initial_state=decoder_input_state)

decoder_states = [state_h,state_c]

decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_input_state,[decoder_outputs] + decoder_states)

In [12]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1,output_voc_size))
    target_seq[0,0,output_to_num['\t']] = 1.
    stop_condition = False
    decode_sentence = ''
    while not stop_condition:
        output_tokens,h,c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0,0,:])
        
        sampled_char = output_to_char[sampled_token_index]
        decode_sentence+=sampled_char
        
        if(sampled_char == '\n' or len(decode_sentence) > output_max_seq):
            stop_condition = True
        
        target_seq = np.zeros((1,1,output_voc_size))
        target_seq[0,0,output_to_num[sampled_char]] = 1.
        states_value = [h,c]
    
    return decode_sentence

In [13]:
input_to_char = {ii:nn for ii,nn in enumerate(input_voc)}
output_to_char = {ii:nn for ii,nn in enumerate(output_voc)}

In [41]:
for i in range(20):
    input_seq = encoder_input_datas[i:i+1]
    print("입력 : ", input_data[i])
    print("정답 : ", output_data[i][1:-1])
    print("모델 : ", decode_sequence(input_seq))

입력 :  Go.
정답 :  Va !
모델 :  Continuez !

입력 :  Hi.
정답 :  Salut !
모델 :  Salut.

입력 :  Hi.
정답 :  Salut.
모델 :  Salut.

입력 :  Run!
정답 :  Cours !
모델 :  Coupe !

입력 :  Run!
정답 :  Courez !
모델 :  Coupe !

입력 :  Who?
정답 :  Qui ?
모델 :  Qui s'est passée ?

입력 :  Wow!
정답 :  Ça alors !
모델 :  Dis le main.

입력 :  Fire!
정답 :  Au feu !
모델 :  Pas de mon !

입력 :  Help!
정답 :  À l'aide !
모델 :  À l'argent !

입력 :  Jump.
정답 :  Saute.
모델 :  Fais-en une manière.

입력 :  Stop!
정답 :  Ça suffit !
모델 :  Arrêtez !

입력 :  Stop!
정답 :  Stop !
모델 :  Arrêtez !

입력 :  Stop!
정답 :  Arrête-toi !
모델 :  Arrêtez !

입력 :  Wait!
정답 :  Attends !
모델 :  Attendez !

입력 :  Wait!
정답 :  Attendez !
모델 :  Attendez !

입력 :  Go on.
정답 :  Poursuis.
모델 :  Va chercher !

입력 :  Go on.
정답 :  Continuez.
모델 :  Va chercher !

입력 :  Go on.
정답 :  Poursuivez.
모델 :  Va chercher !

입력 :  Hello!
정답 :  Bonjour !
모델 :  À l'argent !

입력 :  Hello!
정답 :  Salut !
모델 :  À l'argent !

